<div style="text-align: center; background-color: #b1d1ff; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 20px; font-size: 56px; font-weight: bold; border-radius: 0 0 0 0; box-shadow: 0px 6px 8px rgba(0, 0, 0, 0.2);">Preprocessing</div>

***Notebook này được thiết kế để thực hiện các nhiệm vụ tiền xử lí dữ liệu***
<br>*Trong các phần sắp tới, chúng ta sẽ thực hiện một loạt các bước tiền xử lý dữ liệu, như chuyển đổi kiểu dữ liệu, điền giá trị cho các giá trị bị thiếu, và nhiều hơn nữa, để cải thiện chất lượng và khả năng sử dụng tổng thể của tập dữ liệu.*

# **Import**

### **Thư viện**

*Khai báo các thư viện sử dụng trong notebook*

In [21]:
import numpy as np
import pandas as pd
import re

### **Dữ liệu**

*Đọc file dữ liệu*

In [22]:
file_path = "data/data_cleaned_2021.csv"
df = pd.read_csv(file_path)

*Nhìn 5 dòng đầu tiên của dữ liệu để có cái nhìn tổng quan nhất*

In [23]:
df.head()

,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,...,tensor,hadoop,tableau,bi,flink,mongo,google_an,job_title_sim,seniority_by_title,Degree
0,0,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEducation Required: Bachelor’s degree required, preferably in math, engineering, business, or the sciences.\nSkills Required:\nBachelor’s Degree in relev...",3.80,Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",501 - 1000,1973,...,0,0,1,1,0,0,0,data scientist,na,M
1,1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe Healthcare Data Scientist position will join our Advanced Analytics group at the University of Maryland Medical System (UMMS) in support of its strat...,3.40,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",10000+,1984,...,0,0,0,0,0,0,0,data scientist,na,M
2,2,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information security company. We are the world's largest provider of new-school security awareness training and simulated phishing. KnowBe4 was created to help organ...",4.80,KnowBe4\n4.8,"Clearwater, FL","Clearwater, FL",501 - 1000,2010,...,0,0,0,0,0,0,0,data scientist,na,M
3,3,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\nDirectorate: Earth & Biological Sciences\n\nDivision: Biological Sciences\n\nGroup: Exposure Science Team\n*Job Description**\nThe Biological System S...,3.80,PNNL\n3.8,"Richland, WA","Richland, WA",1001 - 5000,1965,...,0,0,0,0,0,0,0,data scientist,na,na
4,4,Data Scientist,$86K-$143K (Glassdoor est.),"Data Scientist\nAffinity Solutions / Marketing Cloud seeks smart, curious, technically savvy candidates to join our cutting-edge data science team. We hire the best and brightest and give them the...",2.90,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 - 200,1998,...,0,0,0,0,0,0,0,data scientist,na,na


# **Xử lý data**

### **Dữ liệu có bao nhiêu cột và dòng?**

In [24]:
shape = None
shape = df.shape
print("- Số dòng:", shape[0])
print("- Số cột:", shape[1])

if shape[0] > 1000:
    print(f"- Dữ liệu tốt.")
else:
    print(f"- Dữ liệu chưa tốt.")

- Số dòng: 742
- Số cột: 42
- Dữ liệu chưa tốt.


- Ta thấy, dữ liệu có 42 cột và 742 dòng.
- Nhìn chung, đây là một bộ dữ liệu khá nhỏ, không có hơn 1000 để đáp ứng cái tiêu chí ban đầu của thầy, tuy nhiên nhóm chúng em muốn khai thác chủ đề về mức lương của các `Data Scientist` - một ngành nghề mà chúng em đang theo đuổi.  

### **Dữ liệu có bị trùng lặp không?**

In [25]:
num_duplicated_rows = None
# Kiểm tra các dòng trùng lặp
num_duplicated_rows = df.duplicated().sum()

In [26]:
if num_duplicated_rows == 0:
    print(f"- Dữ liệu không có dòng trùng lặp.")
else:
    print(f"- Dữ liệu có {num_duplicated_rows} dòng trùng lặp.")

- Dữ liệu không có dòng trùng lặp.


Như chúng ta có thể thấy, không có bất kỳ dòng nào bị trùng lặp. Đây cũng có thể là lí do mà bộ dữ liệu này khá ít dòng, chỉ có 742 dòng.

### **Mỗi dòng có ý nghĩa gì? Có vấn đề gì nếu các dòng có ý nghĩa khác nhau?**

- Mỗi dòng trong bộ dữ liệu là thông tin về một tin tuyển dụng liên quan đến vị trí `Nhà Khoa học dữ liệu (Data Scientist)` từ `www.glassdoor.com` ở Hoa Kỳ, bao gồm các thông tin quan trọng như loại vị trí, mức lương dự tính và rating. 
- Sự khác biệt giữa các dòng vô cùng quan trọng vì nó phản ánh `sự đa dạng của thị trường tuyển dụng` Data Scientist.

### **Mỗi cột có ý nghĩa gì?**

- **Job Title**: Loại vị trí tuyển dụng.
- **Salary Estimate**: Mức lương dự tính (USD/năm) và nguồn thông tin.
- **Job Description**: Mô tả công việc cho vị trí.
- **Rating**: Chỉ số thể hiện đánh giá của công ty (thang 5).
- **Company Name**: Tên công ty.
- **Location**: Địa chỉ cho công việc.
- **Headquarters**: Địa chỉ trụ sở chính của công ty.
- **Size**: Số nhân viên làm việc tại công ty.
- **Founded**: Năm thành lập.
- **Type of ownership**: Công ty thuộc sở hữu tư nhân, công cộng hay của nhà nước.
- **Industry**: Thuộc ngành.
- **Sector**: Thuộc lĩnh vực.
- **Revenue**: Tổng doanh thu của công ty mỗi năm.
- **Competitors**: Đối thủ cạnh tranh hiện tại của công ty.
- **Lower Salary**: Mức lương tối thiểu được báo cáo cho công việc ở một công ty cụ thể.
- **Upper Salary**: Mức lương tối đa được báo cáo cho công việc ở một công ty cụ thể.
- **Avg Salary(K)**: Mức lương trung bình được báo cáo cho công việc ở một công ty cụ thể.
- **company_txt**: Tên công ty (không chứa số).
- **Job Location**: Địa chỉ bang cho công việc.
- **Age**: Tuổi đời của công ty
- **job_title_sim**: Chức danh đơn giản.
- **seniority_by_title**: Thâm niên trong chức danh.
- **Degree**: Yêu cầu số năm kinh nghiệm trong vị trí.
- **Hourly**: Lương có được báo cáo theo giờ hay không.
- **Employer provided**: Lương do người sử dụng lao động cung cấp hay không.
- **Python**: Có kỹ năng về Python hay không.
- **spark**: Có kỹ năng về Spark hay không.
- **aws**: Có kỹ năng về AWS hay không.
- **excel**: Có kỹ năng về Excel hay không.
- **sql**: Có kỹ năng về SQL hay không.
- **sas**: Có kỹ năng về SAS hay không.
- **keras**: Có kỹ năng về Keras hay không.
- **pytorch**: Có kỹ năng về Pytorch hay không.
- **scikit**: Có kỹ năng về Scikit hay không.
- **tensor**: Có kỹ năng về Tensor hay không.
- **hadoop**: Có kỹ năng về Hadoop hay không.
- **tableau**: Có kỹ năng về Tableau hay không.
- **bi**: Có kỹ năng về PowerBI hay không.
- **flink**: Có kỹ năng về Flink hay không.
- **mongo**: Có kỹ năng về MongoDB hay không.
- **google_an**: Có kỹ năng về phân tích Google hay không.

### **Có bao nhiêu giá trị bị thiếu?**

*Chúng ta cần kiểm tra dữ liệu trong các cột của tập dữ liệu.*

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 742 entries, 0 to 741
Data columns (total 42 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   index               742 non-null    int64  
 1   Job Title           742 non-null    object 
 2   Salary Estimate     742 non-null    object 
 3   Job Description     742 non-null    object 
 4   Rating              742 non-null    float64
 5   Company Name        742 non-null    object 
 6   Location            742 non-null    object 
 7   Headquarters        742 non-null    object 
 8   Size                742 non-null    object 
 9   Founded             742 non-null    int64  
 10  Type of ownership   742 non-null    object 
 11  Industry            742 non-null    object 
 12  Sector              742 non-null    object 
 13  Revenue             742 non-null    object 
 14  Competitors         742 non-null    object 
 15  Hourly              742 non-null    int64  
 16  Employer

### **Các cột có kiểu dữ liệu gì?**

In [28]:
df.dtypes

index                   int64
Job Title              object
Salary Estimate        object
Job Description        object
Rating                float64
Company Name           object
Location               object
Headquarters           object
Size                   object
Founded                 int64
Type of ownership      object
Industry               object
Sector                 object
Revenue                object
Competitors            object
Hourly                  int64
Employer provided       int64
Lower Salary            int64
Upper Salary            int64
Avg Salary(K)         float64
company_txt            object
Job Location           object
Age                     int64
Python                  int64
spark                   int64
aws                     int64
excel                   int64
sql                     int64
sas                     int64
keras                   int64
pytorch                 int64
scikit                  int64
tensor                  int64
hadoop    

Nhìn chung, ở bộ dữ liệu này, các cột chủ yếu có kiểu dữ liệu `int64` và `object`, kiểu dữ liệu `object` ở bộ dữ liệu này nhìn chung cũng chủ yếu là kiểu `string`.

### **Hiệu chỉnh dữ liệu trong các cột như thế nào?**

In [29]:
# Lấy 2 chỉ số của mức lương dự tính
df['Salary Estimate'] = df['Salary Estimate'].apply(lambda x: x.split('(')[0])
df['Salary Estimate'] = df['Salary Estimate'].apply(lambda x: x.replace("K", "").replace("$", ""))
df['Salary Estimate'] = df['Salary Estimate'].apply(lambda x: x.split(':')[1] if ':' in x else x)
df['Salary Estimate'] = df['Salary Estimate'].apply(lambda x: x.split(' P')[0] if ' P' in x else x)

# Xác định xem địa chỉ công ty và trụ sở có cùng một thành phố hay không
df['Same City'] = df.apply(lambda x: 1 if x.Location == x.Headquarters else 0, axis = 1)

# Kiểm tra mức lương dự tính
df['Lower Salary'] =df['Salary Estimate'].apply(lambda x: int(x.split("-")[0]))
df['Upper Salary'] = df['Salary Estimate'].apply(lambda x: int(x.split("-")[1]))
df['Avg Salary(K)'] = (df['Lower Salary'] + df['Upper Salary']) / 2

# Sửa tên công ty
df['Company Name'] = df['Company Name'].apply(lambda x: x.split("\n")[0])

# Tính lại tuổi đời của công ty
df['Age'] = df['Founded'].apply(lambda x: x if x < 1 else 2024 - x)

# Ở cột 'Competitors', các hàng có giá trị -1, sẽ được chuyển về None
df['Competitors'] = df['Competitors'].replace(-1, None)

df

,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,...,hadoop,tableau,bi,flink,mongo,google_an,job_title_sim,seniority_by_title,Degree,Same City
0,0,Data Scientist,53-91,"Data Scientist\nLocation: Albuquerque, NM\nEducation Required: Bachelor’s degree required, preferably in math, engineering, business, or the sciences.\nSkills Required:\nBachelor’s Degree in relev...",3.80,Tecolote Research,"Albuquerque, NM","Goleta, CA",501 - 1000,1973,...,0,1,1,0,0,0,data scientist,na,M,0
1,1,Healthcare Data Scientist,63-112,What You Will Do:\n\nI. General Summary\n\nThe Healthcare Data Scientist position will join our Advanced Analytics group at the University of Maryland Medical System (UMMS) in support of its strat...,3.40,University of Maryland Medical System,"Linthicum, MD","Baltimore, MD",10000+,1984,...,0,0,0,0,0,0,data scientist,na,M,0
2,2,Data Scientist,80-90,"KnowBe4, Inc. is a high growth information security company. We are the world's largest provider of new-school security awareness training and simulated phishing. KnowBe4 was created to help organ...",4.80,KnowBe4,"Clearwater, FL","Clearwater, FL",501 - 1000,2010,...,0,0,0,0,0,0,data scientist,na,M,1
3,3,Data Scientist,56-97,*Organization and Job ID**\nJob ID: 310709\n\nDirectorate: Earth & Biological Sciences\n\nDivision: Biological Sciences\n\nGroup: Exposure Science Team\n*Job Description**\nThe Biological System S...,3.80,PNNL,"Richland, WA","Richland, WA",1001 - 5000,1965,...,0,0,0,0,0,0,data scientist,na,na,1
4,4,Data Scientist,86-143,"Data Scientist\nAffinity Solutions / Marketing Cloud seeks smart, curious, technically savvy candidates to join our cutting-edge data science team. We hire the best and brightest and give them the...",2.90,Affinity Solutions,"New York, NY","New York, NY",51 - 200,1998,...,0,0,0,0,0,0,data scientist,na,na,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
737,950,"Sr Scientist, Immuno-Oncology - Oncology",58-111,"Site Name: USA - Massachusetts - Cambridge\nPosted Date: Mar 24 2020\n\nAre you energized by a challenging role in immuno-oncology, where scientific demand is driving team growth? If so, this Seni...",3.90,GSK,"Cambridge, MA","Brentford, United Kingdom",10000+,1830,...,0,0,0,0,0,0,other scientist,sr,M,0
738,951,Senior Data Engineer,72-133,"THE CHALLENGE\nEventbrite has a world-class data repository of live events, powering millions of events and hundreds of millions of ticket transactions each year in 170+ countries. Our platform al...",4.40,Eventbrite,"Nashville, TN","San Francisco, CA",1001 - 5000,2006,...,1,0,0,0,0,0,data engineer,sr,na,0
739,952,"Project Scientist - Auton Lab, Robotics Institute",56-91,The Auton Lab at Carnegie Mellon University is a large academic group driven by a desire to make a real-world difference in a broad range of research interests. The areas of our current focus incl...,2.60,Software Engineering Institute,"Pittsburgh, PA","Pittsburgh, PA",501 - 1000,1984,...,0,0,0,0,0,0,other scientist,na,P,1
740,953,Data Science Manager,95-160,Data Science ManagerResponsibilities:\n\nOversee a team of Data Scientists and Data Visualization Analysts who transform enterprise data into value drive insights\n\nDesign and implement processes...,3.20,"Numeric, LLC","Allentown, PA","Chadds Ford, PA",1 - 50,-1,...,0,0,0,0,0,0,data scientist,na,na,0


### **Xử lý các cột có giá trị bị thiếu cao hoặc các cột không cần thiết?**



Để tăng tính rõ ràng, chúng ta sẽ hiển thị danh sách các cột có tỷ lệ giá trị bị thiếu đáng kể.

In [30]:
# Tính tỷ lệ giá trị bị thiếu cho mỗi cột
missing_ratio = df.isnull().mean() * 100

# Lọc ra các cột có tỷ lệ giá trị bị thiếu lớn hơn 60%
missing_column = missing_ratio[missing_ratio > 60]

# In ra tên cột và tỷ lệ giá trị bị thiếu tương ứng
for column_name, missing_percent in missing_column.items():
    print(f"'{column_name}': {missing_percent}%")

- Wow, ta có thể thấy rằng, không có bất kì cột nào có giá trị bị thiếu lớn hơn 60%.
- Do đó, bộ dữ liệu này ta không cần xử lí các cột có giá trị bị thiếu cao. 

Tiếp theo, nhìn vào các cột, nhóm thấy, cột `index` chắc chắn sẽ không được sử dụng để phân tích, vì nó chỉ có chức năng thể hiện số thứ tự. Do đó, ta sẽ xóa cột `index`.

In [31]:
del df['index']

Nhìn sâu xa hơn về bộ dữ liệu, ta có thể thấy, cột `Job Description` được sử dụng để mô tả công việc, tuy nhiên cột này, chúng ta cũng có thể phỏng đoán rằng cột này sẽ không được sử dụng để phân tích. Do đó ta cũng sẽ xóa cột `Job Description`.

In [32]:
del df['Job Description']

### **Với mỗi cột có kiểu dữ liệu số, các giá trị được phân bổ như thế nào?**

Đối với các cột có kiểu dữ liệu số, ta sẽ tính:

- The min
- The lower quartile (phân vị 25)
- The median (phân vị 50)
- The upper quartile (phân vị 75)
- The max

In [33]:
num_col_info_df = None
num_col_info_df = df.select_dtypes(include=['number'])

# Hàm tính lower quartile (phân vị 25)
def lower_quartile(x):
    return x.quantile(0.25)

# Hàm tính median (phân vị 50)
def median(x):
    return x.quantile(0.5)

# Hàm tính upper quartile (phân vị 75)
def upper_quartile(x):
    return x.quantile(0.75)

num_col_info_df = num_col_info_df.agg(["min", lower_quartile, median, upper_quartile, "max"])
num_col_info_df = num_col_info_df.round(1)
num_col_info_df

,Rating,Founded,Hourly,Employer provided,Lower Salary,Upper Salary,Avg Salary(K),Age,Python,spark,...,pytorch,scikit,tensor,hadoop,tableau,bi,flink,mongo,google_an,Same City
min,-1.00,-1.00,0.00,0.00,10.00,16.00,13.50,-1.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
lower_quartile,3.30,1939.00,0.00,0.00,52.00,96.00,73.50,15.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
median,3.70,1988.00,0.00,0.00,69.50,124.00,97.50,28.00,1.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00
upper_quartile,4.00,2007.00,0.00,0.00,91.00,155.00,122.50,63.00,1.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00
max,5.00,2019.00,1.00,1.00,202.00,306.00,254.00,280.00,1.00,1.00,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00


### **Xử lí trường hợp ngoại lệ**

#### Tính giá trị thống kê để đánh giá trường hợp ngoại lệ

In [34]:
# Tính các giá trị thống kê cơ bản của các cột
basic_stats = num_col_info_df.describe()
pd.set_option('display.float_format', '{:.2f}'.format)
basic_stats

,Rating,Founded,Hourly,Employer provided,Lower Salary,Upper Salary,Avg Salary(K),Age,Python,spark,...,pytorch,scikit,tensor,hadoop,tableau,bi,flink,mongo,google_an,Same City
count,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,...,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00
mean,3.00,1590.40,0.20,0.20,84.90,139.40,112.20,77.00,0.60,0.20,...,0.20,0.20,0.20,0.20,0.20,0.20,0.20,0.20,0.20,0.60
std,2.32,890.14,0.45,0.45,71.90,106.47,88.98,115.90,0.55,0.45,...,0.45,0.45,0.45,0.45,0.45,0.45,0.45,0.45,0.45,0.55
min,-1.00,-1.00,0.00,0.00,10.00,16.00,13.50,-1.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,3.30,1939.00,0.00,0.00,52.00,96.00,73.50,15.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
50%,3.70,1988.00,0.00,0.00,69.50,124.00,97.50,28.00,1.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00
75%,4.00,2007.00,0.00,0.00,91.00,155.00,122.50,63.00,1.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00
max,5.00,2019.00,1.00,1.00,202.00,306.00,254.00,280.00,1.00,1.00,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00


#### Ngoại lệ được xử lý như thế nào?

- Ta sẽ sử dụng phương pháp Z-score để tìm ra các giá trị ngoại lệ.
- Nhìn vào các cột, và mục đích phân tích, ta sẽ đánh mạnh, tập trung xử lí ngoại lệ ở cột `Avg Salary(K)`

In [35]:
# Tính trung bình và độ lệch chuẩn của cột Salary
mean_salary = df['Avg Salary(K)'].mean()
std_salary = df['Avg Salary(K)'].std()

# Xác định ngưỡng Z-score
threshold = 2

# Tính Z-score cho từng giá trị lương
df['Z-score'] = (df['Avg Salary(K)'] - mean_salary) / std_salary

# Lọc ra các giá trị ngoại lệ
outliers = df[abs(df['Z-score']) > threshold]

# In ra các giá trị ngoại lệ
print("Các giá trị ngoại lệ:")
print(outliers[['Job Title', 'Avg Salary(K)', 'Z-score']])

Các giá trị ngoại lệ:
                                                  Job Title  Avg Salary(K)  \
46                                      Lead Data Scientist         184.50   
71                                   Director, Data Science         179.50   
103                                   Senior Data Scientist         237.50   
165                                   Medical Lab Scientist          20.50   
176  Principal Data Scientist with over 10 years experience         225.00   
193                         Staff Machine Learning Engineer         181.00   
195                                      Lead Data Engineer         205.00   
197                            Medical Laboratory Scientist          21.50   
210    Medical Technologist / Clinical Laboratory Scientist          20.00   
240                                          Data Scientist          15.50   
246                                   Medical Lab Scientist          20.50   
266  Principal Data Scientist with over 10

Bởi vì bộ dữ liệu chỉ có 742 dòng, do đó, ta sẽ không xóa các dòng có ngoại lệ ở trên, mà ta sẽ thay thế các ngoại lệ đó thành giá trị trung bình của cột đó.

In [36]:
# Thay thế giá trị ngoại lệ bằng giá trị trung bình lương
df.loc[abs(df['Z-score']) > threshold, 'Avg Salary(K)'] = mean_salary

# In ra thông báo thành công
print("Các giá trị ngoại lệ đã được thay thế bằng giá trị trung bình lương.")

Các giá trị ngoại lệ đã được thay thế bằng giá trị trung bình lương.


Xem lại cột `Avg Salary(K)` sau khi đã xử lí ngoại lệ.

In [37]:
df['Avg Salary(K)']

0      72.00
1      87.50
2      85.00
3      76.50
4     114.50
       ...  
737    84.50
738   102.50
739    73.50
740   127.50
741    93.50
Name: Avg Salary(K), Length: 742, dtype: float64

### **Với mỗi cột có kiểu dữ liệu không phải số, các giá trị được phân bổ như thế nào?**

Đối với các cột có kiểu dữ liệu không phải số, ta sẽ tính:

- Số lượng giá trị
- Tỷ lệ phần trăm (từ 0 đến 100) của mỗi giá trị được sắp xếp theo tỷ lệ phần trăm giảm dần

In [38]:
pd.set_option('display.max_colwidth', 200)

cat_col_info_df = df.select_dtypes(exclude=['number'])
cat_col_info_df

# Hàm để tính số lượng giá trị duy nhất
def num_values(x):
    return len(x.dropna().unique())

# Hàm để tính các tỷ lệ giá trị
def value_ratios(x):
    # Tạo một Series với tỷ lệ xuất hiện của từng giá trị
    value_counts = x.value_counts(normalize=True)

    # Sắp xếp theo tỷ lệ giảm dần
    sorted_value_counts = value_counts.sort_values(ascending=False)
    
    # Lưu vào từ điển
    value_ratios = {value: round(percentage * 100, 1) for value, percentage in sorted_value_counts.items()}
    
    return value_ratios

# Gán DataFrame mới để lưu kết quả
cat_col_info_df = cat_col_info_df.agg([num_values, value_ratios])
cat_col_info_df

,Job Title,Salary Estimate,Company Name,Location,Headquarters,Size,Type of ownership,Industry,Sector,Revenue,Competitors,company_txt,Job Location,job_title_sim,seniority_by_title,Degree
num_values,264,416,343,200,198,8,9,60,25,13,128,343,37,10,3,3
value_ratios,"{'Data Scientist': 17.7, 'Data Engineer': 7.1, 'Senior Data Scientist': 4.6, 'Data Analyst': 2.0, 'Senior Data Engineer': 1.9, 'Senior Data Analyst': 1.6, 'Lead Data Scientist': 1.1, 'Marketing Da...","{'49-113 ': 0.8, '54-115 ': 0.8, '21-34': 0.8, '86-143 ': 0.8, '74-124 ': 0.7, '76-142 ': 0.7, '107-173 ': 0.7, '81-167 ': 0.7, '44-78 ': 0.5, '44-86 ': 0.5, '35-62 ': 0.5, '64-106 ': 0.5, '39-66 ...","{'MassMutual': 1.9, 'Takeda Pharmaceuticals': 1.9, 'Reynolds American': 1.9, 'Software Engineering Institute': 1.5, 'PNNL': 1.3, 'Liberty Mutual Insurance': 1.3, 'AstraZeneca': 1.2, 'MITRE': 1.1, ...","{'New York, NY': 7.4, 'San Francisco, CA': 6.6, 'Cambridge, MA': 6.3, 'Chicago, IL': 4.3, 'Boston, MA': 3.1, 'San Jose, CA': 1.8, 'Pittsburgh, PA': 1.6, 'Washington, DC': 1.5, 'Rockville, MD': 1.5...","{'New York, NY': 7.0, 'San Francisco, CA': 5.7, 'Chicago, IL': 4.0, 'Cambridge, MA': 2.7, 'Springfield, MA': 1.9, 'Boston, MA': 1.9, 'Winston-Salem, NC': 1.9, 'OSAKA, Japan': 1.9, 'Richland, WA': ...","{'1001 - 5000 ': 20.2, '501 - 1000 ': 18.1, '10000+ ': 17.5, '201 - 500 ': 15.8, '51 - 200 ': 12.7, '5001 - 10000 ': 10.2, '1 - 50 ': 4.2, 'unknown': 1.3}","{'Company - Private': 55.3, 'Company - Public': 26.0, 'Nonprofit Organization': 7.4, 'Subsidiary or Business Segment': 4.6, 'Government': 2.0, 'Hospital': 2.0, 'College / University': 1.8, 'Other ...","{'Biotech & Pharmaceuticals': 15.1, 'Insurance Carriers': 8.5, 'Computer Hardware & Software': 8.0, 'IT Services': 6.7, 'Health Care Services & Hospitals': 6.6, 'Enterprise Software & Network Solu...","{'Information Technology': 24.3, 'Biotech & Pharmaceuticals': 15.1, 'Business Services': 13.1, 'Insurance': 9.3, 'Health Care': 6.6, 'Finance': 5.7, 'Manufacturing': 4.6, 'Aerospace & Defense': 3....","{'Unknown / Non-Applicable': 27.5, '$10+ billion (USD)': 16.7, '$100 to $500 million (USD)': 12.3, '$1 to $2 billion (USD)': 8.1, '$500 million to $1 billion (USD)': 7.7, '$50 to $100 million (USD...","{'-1': 62.0, 'Novartis, Baxter, Pfizer': 1.9, 'Oak Ridge National Laboratory, National Renewable Energy Lab, Los Alamos National Laboratory': 1.6, 'Travelers, Allstate, State Farm': 1.3, 'Roche, G...","{'MassMutual': 1.9, 'Takeda Pharmaceuticals': 1.9, 'Reynolds American': 1.9, 'Software Engineering Institute': 1.5, 'PNNL': 1.3, 'Liberty Mutual Insurance': 1.3, 'AstraZeneca': 1.2, 'MITRE': 1.1, ...","{'CA': 20.5, 'MA': 13.9, 'NY': 9.7, 'VA': 5.5, 'IL': 5.4, 'MD': 4.7, 'PA': 4.4, 'TX': 3.8, 'WA': 2.8, 'NC': 2.8, 'NJ': 2.3, 'FL': 2.2, 'OH': 1.9, 'TN': 1.8, 'DC': 1.5, 'CO': 1.5, 'UT': 1.3, 'IN': ...","{'data scientist': 42.2, 'other scientist': 19.3, 'data engineer': 16.0, 'analyst': 13.6, 'machine learning engineer': 3.0, 'Data scientist project manager': 2.2, 'na': 1.3, 'data analitics': 1.1,...","{'na': 69.9, 'sr': 29.6, 'jr': 0.4}","{'na': 51.6, 'M': 34.0, 'P': 14.4}"


## **Lưu dữ liệu đã được làm sạch**

Cuối cùng, chúng ta sẽ lưu dữ liệu đã được tiền xử lí (làm sạch, xử lí ngoại lệ) vào một file .csv có tên là cleaned_data_DS_Salary để thuận lợi cho các việc phân tích sau này.

In [39]:
df.to_csv('data/cleaned_data_DS_Salary.csv', index = False)